In [ ]:
!pip install uwsgi
import flask
from flask import request
from keras.models import load_model
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from flask import jsonify
import os
!pip install facenet-pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import requests
from PIL import Image
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
import io
# from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive
!ls

workers = 0 if os.name == 'nt' else 4

Mounted at /content/drive
/content/drive/My Drive
 0.jpg
'?????????????? ?? ???????????????????? ???????? ?????????????????? ?????????????? ???????????? 2.pdf'
'40 USD.jpg'
'Ali_Eshtehardian_CV (1).pdf'
 Ali_Eshtehardian_CV.pdf
'Ali Eshtehardian.mp4'
 autocar_control.py
'Banking challenges can be solved by data science:.gdoc'
'Berkeley Personal Statement.gdoc'
'Berkeley SOP.gdoc'
 bert_Q2.ipynb
'Biometric datasets.gsheet'
 CelebA_Spoof.zip.001
 cifar
'Colab Notebooks'
'Copy of img_celeba.7z'
'cover letter.gdoc'
 Cuda-TPU.mp4
 C-VAE2.ipynb
 C-VAE.ipynb
 DAE.ipynb
 daniel_craig.jpg
 data
 Datamining_proposal.gdoc
 DDPG-PyTorch-master
'Deep RL Brief.zip'
'Delft motivation letter 2.gdoc'
'Delft Motivation letter 3.gdoc'
'Delft motivation letter4.gdoc'
'Delft motivation letter 5.gdoc'
'Delft-RL in real world.gdoc'
 densenet.ipynb
 densenet_torch.ipynb
 Eshtehardian-abstract.pdf
'Eshtehardian - cover letter.gdoc'
'Eshtehardian_CV_2021 (1).pdf'
'Eshtehardian_CV_2021 (2).pdf'
 Eshtehardian_CV_

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://5m35vsj9art-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
# initialize our Flask application and the model
app = flask.Flask(__name__)
# run_with_ngrok(app)
model = None

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

# resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def load_model():
    # load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def prepare_image(image, target):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # return the processed image
    return image

def collate_fn(x):
    return x[0]


def classify_image(image):

  resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
  dataset = datasets.ImageFolder('./Polwinno_pics')
  dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
  loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers, batch_size=1)
  aligned = []
  names = []
  for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])
  input, prob = mtcnn(image, return_prob=True)
  if x_aligned is not None:
    aligned.append(input)
    names.append('unknown')

  alignedt = torch.stack(aligned).to(device)
  embeddings = resnet(alignedt).detach().cpu()

  dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]


  data=pd.DataFrame(dists, columns=names, index=names)
  labels0=dict(data['unknown'])
  keys=labels0.keys()
  values=labels0.values()
  # print(labels0)
  # print(keys)
  # print(values)
  # def get_key(val):
  #     for key, value in my_dict.items():
  #         if val == value:
  #             return key

  #     return "key doesn't exist"
  labels=list(values)
  labels.sort()
  # print(labels)



  # if labels[1]<1:
  #   print(list(keys)[list(values).index(labels[1])])
  # else:
  #   print('unknown')
  if labels[1]<1:
    return list(keys)[list(values).index(labels[1])]


  else:
    return 'unknown'



In [ ]:
@app.route("/", methods=["POST"])

def predict():

  # initialize the data dictionary that will be returned from the
  # view
  data = {"success": False}

  # ensure an image was properly uploaded to our endpoint
  if request.method == "POST":
      if request.files.get("image"):
          # read the image in PIL format
          image = request.files["image"].read()
          image = Image.open(io.BytesIO(image))
          print(type(image))
          # preprocess the image and prepare it for classification
          # image = prepare_image(image, target=(224, 224))



          # classify the input image and then initialize the list
          # of predictions to return to the client
          preds = classify_image(image)
          data=preds
          # results = imagenet_utils.decode_predictions(preds)
          # data["predictions"] = []

          # # indicate that the request was a success
          # data["success"] = True

  # return the data dictionary as a JSON response
  return flask.jsonify(data)

In [ ]:
!pip install pyOpenSSL
import OpenSSL
from OpenSSL import SSL


context = SSL.Context(SSL.TLSv1_2_METHOD)
# context.use_privatekey_file('server.key')
# context.use_certificate_file('server.crt')

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 3.6 MB 27.9 MB/s 


In [ ]:
# if this is the main thread of execution first load the model and
# then start the server

if __name__ == "__main__":
    print(("* Loading the model and Flask starting server..."
        "please wait until server has fully started"))
    load_model()
    app.run(host='0.0.0.0', debug=True, ssl_context=context)


* Loading the model and Flask starting server...please wait until server has fully started


  0%|          | 0.00/107M [00:00<?, ?B/s]

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on https://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
